In [12]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "tinyllama"
#MODEL = "my-custom-tinyllama"

##langsmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)



In [13]:
#langchain to load chroma db data
from langchain_community.embeddings import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
from langchain.vectorstores import Chroma
chroma_db = Chroma(persist_directory="./chroma_db", 
                   embedding_function=embed_model,
                   collection_name="DB_collection")
retriever = chroma_db.as_retriever(search_kwargs={"k": 5})
retriever.invoke(input="who is aparna?")

[Document(page_content='Naresh Dulam – born and brought up in village Burugupally, District Karimnagar, Telangana state.  \nHe completed his schooling in his village private school in Telugu medium. He had scored 534 marks  \nin his 10th class and joined for Master Jr. College for intermediate and completed it in 2006 with  \nmarks of 955 and EMCET rank of 17k. He then joined for bachelor’s degree in Jyothismathi engineering \ncollege, Karimnagar, passed with  \n69%.  \nAfter the engineering, he joined Infor global  private  ltd as R&D engineer in Supply Chain Execution  \nproduct development, worked as part COE team to product enhancements and maintenance.   \nLater he joined in Oracle India private  ltd as Application Developer in developing product of Fusion GRC  \nfor 16 months before he moved to USA.   \nIn USA worked as big data analytics at various roles in T -Mobile, ATT, Bank of America and JP  \nMorgan chase.   \nHe married Aparna Balagoni and they have two children of Meenak

In [14]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000020A72B261D0>, search_kwargs={'k': 5})

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.globals import set_debug
from langchain.prompts import PromptTemplate
set_debug(True)

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".
Context: {context}
Question: {input}
"""
prompt = PromptTemplate.from_template(template)


combine_docs_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

#Invoke the retrieval chain
#input = "Where is naresh living currently?"
#input = "What is Naresh's wife name?"
#input = "who is aparna?"
#input = "How may kids naresh have?"
#input = "How many people in naresh's family?"
inputs = ["What are the certification naresh have?"]
#inputs = ["Where is naresh living currently?","What are the certification naresh have?","What is Naresh's wife name?","who is aparna?","How may kids naresh have?"]
for input in inputs:
    response=retrieval_chain.invoke({"input":input})
    print(input + " ====> "+ response["answer"])

#Print the answer to the question
#print(response["answer"])



[chain/start] [1:chain:retrieval_chain] Entering Chain run with input:
{
  "input": "What are the certification naresh have?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "What are the certification naresh have?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "What are the certification naresh have?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents] Entering Chain run with input:
{
  "input": "What are the certification naresh have?"
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents > 5:chain:RunnableLambda] Entering Chain run with input:
{
  "input": "What are the certification naresh have?"
}
[chain/end] [1:chain:retrieval_ch

In [16]:
!pip freeze > requirements.txt